In [5]:
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
import scipy.linalg as sla

In [2]:
# define lattice and generate hopping matrix with OBC

In [14]:
from pythtb import tb_model
def Tmatrix(t,mu):

    # set model parameters
    lat=[[1.0,0.0],[0.0,1.0]]
    orb=[[0.0,0.0]]

    # create TB model
    my_model=tb_model(2,2,lat,orb)
    
    # set on-site energies
    my_model.set_onsite([-mu])
    # set hopping terms (one for each nearest neighbor)
    my_model.set_hop(t, 0, 0, [1, 0])
    my_model.set_hop(t, 0, 0, [0, 1])
    # create a 4x4 lattice
    Nx, Ny = 4, 4
    my_ham = my_model.cut_piece(Nx,0,glue_edgs=False)
    my_ham = my_ham.cut_piece(Ny,1,glue_edgs=False)
    # print Hamiltonian
    return my_ham._gen_ham()


In [15]:
def ExpM(phi,ExpT,Vdiag):
    # T = exp(-T*dtau)
    return np.diag(np.exp(phi * Vdiag)).dot(ExpT)

In [16]:
def SVD_stablizer_back(A,U,D,V):
    # B = U * D * V
    # B * A = U * D * V * B = U * U1 * D1 * V1 = U2 * D1 * V1
    U1, D, V = LA.svd(np.diag(D).dot(V).dot(A))
    return U.dot(U1), D, V
    
def SVD_stablizer_forward(A,U,D,V):
    # B = U * D * V
    # A * B = A * U * D * V = U1 * D1 * V1 * V = U1 * D1 * V2
    U, D, V1 = LA.svd(A.dot(U.dot(np.diag(D))))
    return U, D, V1.dot(V)

In [17]:
def GF(UL,DL,VL,UR,DR,VR):
    # B(t,0) = UL * DL * VL
    # B(beta,t) = VR * DR * UR Take care of it !!!
    U,D,V = LA.svd(LA.inv(UL.dot(UR))+np.diag(DR).dot(VR.dot(VL)).dot(np.diag(DL)))
    return LA.inv(V.dot(UL)).dot(np.diag(1/D)).dot(LA.inv(UR.dot(U)))

In [18]:
def Stable_Bmatrix(phi,ExpT,Vdiag):
    # use SVD algorithm to calculate the stable B matrix
    # ExpT: the matrix of exp(-dtau * T)
    # Vdiag: Ntau x L matrix with each column being - alpha 
    Ntau, L = phi.shape
    U = np.zeros((Ntau+1,L,L))
    D = np.zeros((Ntau+1,L))
    V = np.zeros((Ntau+1,L,L))

    U[0],D[0],V[0] = LA.svd(np.eye(L))
    for i in range(Ntau):
        U[i+1], D[i+1], V[i+1] = SVD_stablizer_forward(ExpM(phi[i], ExpT, Vdiag),U[i],D[i],V[i])
    return U,D,V

In [19]:
def Update_HS(GFup, GFdn, phi, alpha):
    Ident = np.eye(len(phi))
    for i in range(len(phi)):
        dup = np.exp(-2 * alpha * phi[i]) -1 
        ddn = np.exp(2 * alpha * phi[i]) -1
        Rup = 1 + dup * (1 - GFup[i,i])
        Rdn = 1 + ddn * (1 - GFdn[i,i])
        
        if  np.abs(Rup * Rdn) > np.random.rand():
            
            GFup -= dup * np.outer(GFup[:,i], Ident[i]-GFup[i,:])/Rup
            GFdn -= ddn * np.outer(GFdn[:,i], Ident[i]-GFdn[i,:])/Rdn
            phi[i] = -phi[i]
    
    return GFup, GFdn, phi

In [20]:
def Sweep_backward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha):
    # Input GFup, GFdn should be G(0,0) = G(\beta,\beta)
    # The first member of U, D, V should be the result of I = B(0,0) or B(\beta, \beta)
    Ntau = len(phi) 
    ind_revered = np.roll(np.arange(Ntau+1)[::-1], shift=1)
    for n in range(1,Ntau+1):
        i = ind_revered[n]
        i_1 = ind_revered[n-1]
        if np.mod(n,Nstable) == 0:
            # recompute equal time GF to avoid accumulation of numerical error
            GFup_recomputed = GF(Vup[i_1],Dup[i_1],Uup[i_1],Uup[i],Dup[i],Vup[i])
            GFdn_recomputed = GF(Vdn[i_1],Ddn[i_1],Udn[i_1],Udn[i],Ddn[i],Vdn[i])
            # compare with advanced one to get the accumulated error
            GFup_error = np.max(np.abs(GFup_recomputed - GFup))
            GFdn_error = np.max(np.abs(GFdn_recomputed - GFdn))
            print("Devitation between GF by propagating and by recomputed: ", (GFup_error + GFdn_error)/2)
            GFup = GFup_recomputed
            GFdn = GFdn_recomputed
        
        # Update H-S field and Green function
        GFup, GFdn, phi[i-1] = Update_HS(GFup, GFdn, phi[i-1], alpha)

        # Update SVD decompostion for B matrix
        Bup = ExpM(phi[i-1], ExpT,  Vdiag)
        Bdn = ExpM(phi[i-1], ExpT, -Vdiag)

        # Advance equal time GF using new H-S field
        GFup = LA.inv(Bup).dot(GFup).dot(Bup)
        GFdn = LA.inv(Bdn).dot(GFdn).dot(Bdn)

        # Repalce SVD decomposition of B matrix
        Uup[i], Dup[i], Vup[i] = SVD_stablizer_back(Bup, Uup[i_1], Dup[i_1], Vup[i_1])
        Udn[i], Ddn[i], Vdn[i] = SVD_stablizer_back(Bdn, Udn[i_1], Ddn[i_1], Vdn[i_1])

    return GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi


In [21]:
def Sweep_forward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha):
    # Input GFup, GFdn should be G(0,0) = G(\beta,\beta)
    # The first member of U, D, V should be the result of I = B(0,0) or B(\beta, \beta)

    # reverse the order of U, D, V, and shift B(0,0) to the first one, to make the sweep backward
    
    Ntau = len(phi) 
    for i in range(1,Ntau+1):
        if np.mod(i,Nstable) == 0:
            # recompute equal time GF to avoid accumulation of numerical error
            GFup_recomputed = GF(Vup[i],Dup[i],Uup[i],Uup[i-1],Dup[i-1],Vup[i-1])
            GFdn_recomputed = GF(Vdn[i],Ddn[i],Udn[i],Udn[i-1],Ddn[i-1],Vdn[i-1])

            # compare with advanced one to get the accumulated error
            GFup_error = np.max(np.abs(GFup_recomputed - GFup))
            GFdn_error = np.max(np.abs(GFdn_recomputed - GFdn))
            print("Devitation between GF by propagating and by recomputed: ", (GFup_error + GFdn_error)/2)
            GFup = GFup_recomputed
            GFdn = GFdn_recomputed

        # Advance equal green function first note that phi has not been updated
        Bup = ExpM(phi[i-1], ExpT,  Vdiag)
        Bdn = ExpM(phi[i-1], ExpT, -Vdiag)
        GFup = Bup.dot(GFup).dot(LA.inv(Bup))
        GFdn = Bdn.dot(GFdn).dot(LA.inv(Bdn))
        
        # Update H-S field and Green function
        GFup, GFdn, phi[i-1] = Update_HS(GFup, GFdn, phi[i-1], alpha)

        # Update SVD decompostion for B matrix
        Bup = ExpM(phi[i-1], ExpT,  Vdiag)
        Bdn = ExpM(phi[i-1], ExpT, -Vdiag)

        # Repalce SVD decomposition of B matrix
        Uup[i], Dup[i], Vup[i] = SVD_stablizer_forward(Bup, Uup[i-1], Dup[i-1], Vup[i-1])
        Udn[i], Ddn[i], Vdn[i] = SVD_stablizer_forward(Bdn, Udn[i-1], Ddn[i-1], Vdn[i-1])

    return GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi

In [22]:
L = 4 * 4
Ntau = 40
Nstable = 10
dtau = 0.1
U = 4.0
t = -1.0
mu = 4
beta = dtau * Ntau
alpha = np.arccosh(np.exp(0.5*dtau*U))
T = Tmatrix(t,mu).real
ExpT = sla.expm(-dtau * T)
Vdiag = np.zeros(L)
Vdiag[:] = alpha

In [23]:
Nwarm = 500
Nsweep = 2000
phi = np.random.choice([1, -1], size=(Ntau, L))
#The part of spin up and spin down are not coupled but direct product
Uup,Dup,Vup = Stable_Bmatrix(phi,ExpT,Vdiag)
Udn,Ddn,Vdn = Stable_Bmatrix(phi,ExpT,-Vdiag)
GFup = GF(Vup[0],Dup[0],Uup[0],Uup[-1],Dup[-1],Vup[-1])
GFdn = GF(Vdn[0],Ddn[0],Udn[0],Udn[-1],Ddn[-1],Vdn[-1])
sign  = np.zeros(2 * Nsweep)
Number = np.zeros(2 * Nsweep)
# Warm up
for _ in range(Nwarm):
    GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi = Sweep_backward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha)
    GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi = Sweep_forward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha)

# Sweep
for i in range(Nsweep):
    GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi = Sweep_backward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha)
    
    # Measurement 
    sign[i*2] = np.sign(LA.det(GFup) * LA.det(GFdn))
    Number[i*2] = np.diag(GFup).sum() + np.diag(GFdn).sum()

    GFup, GFdn, Uup, Dup, Vup, Udn, Ddn, Vdn, phi = Sweep_forward(GFup,GFdn,Uup,Dup,Vup,Udn,Ddn,Vdn,phi,ExpT,Vdiag,Nstable,alpha)
    
    # Measurement 
    sign[i*2 + 1] = np.sign(LA.det(GFup) * LA.det(GFdn))
    Number[i*2 + 1] = np.diag(GFup).sum() + np.diag(GFdn).sum()


Devitation between GF by propagating and by recomputed:  4.2675585287810236e-13
Devitation between GF by propagating and by recomputed:  8.942759727181837e-13
Devitation between GF by propagating and by recomputed:  5.250619383723176e-12
Devitation between GF by propagating and by recomputed:  1.0237644065824725e-12
Devitation between GF by propagating and by recomputed:  1.2179545566537442e-12
Devitation between GF by propagating and by recomputed:  6.663378182558688e-12
Devitation between GF by propagating and by recomputed:  7.897293929914895e-13
Devitation between GF by propagating and by recomputed:  1.3296724832301265e-12
Devitation between GF by propagating and by recomputed:  3.9170142823730103e-13
Devitation between GF by propagating and by recomputed:  1.1631667851119687e-12
Devitation between GF by propagating and by recomputed:  5.391207879429372e-12
Devitation between GF by propagating and by recomputed:  4.959227473122496e-13
Devitation between GF by propagating and by re

KeyboardInterrupt: 

In [86]:
np.mean(sign)

0.65

In [87]:
sign

array([ 1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,
        1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,
        1.])

In [88]:
Number[0]/32 

0.11001009238643278

In [91]:
np.mean(Number) 

3.9099010255075455

In [92]:
np.mean(sign * Number) / np.mean(sign)/ 32

0.11774391261163175

In [27]:
from run_DQMC import run

N = 4
dtau = 0.1
Nstable = 10
t = -1.0
Nwarm = 500
Nsweep = 2000

mu = 1.7
Ntau = 10
U = 4.0
# 0 = sign, 1 = electron number
sign, Number = run(N, Ntau, dtau, Nstable, U, t, mu, Nwarm, Nsweep)

Start warm up


Warming up:   0%|          | 0/500 [00:00<?, ?it/s]

Devitation between GF by propagating and by recomputed: 7.996719605940505e-13
Devitation between GF by propagating and by recomputed: 6.13062335272363e-13
Devitation between GF by propagating and by recomputed: 1.2871440024930791e-12
Devitation between GF by propagating and by recomputed: 2.3289599415665663e-12
Devitation between GF by propagating and by recomputed: 2.1133112620974615e-12
Devitation between GF by propagating and by recomputed: 1.2531947051266923e-12
Devitation between GF by propagating and by recomputed: 1.9285506351607307e-12
Devitation between GF by propagating and by recomputed: 2.579793000859372e-12
Devitation between GF by propagating and by recomputed: 5.759379518438523e-13
Devitation between GF by propagating and by recomputed: 6.2388705313054515e-12
Devitation between GF by propagating and by recomputed: 6.602253466159169e-13
Devitation between GF by propagating and by recomputed: 4.557076938027649e-12
Devitation between GF by propagating and by recomputed: 7.0

Sweep:   0%|          | 0/2000 [00:00<?, ?it/s]

Devitation between GF by propagating and by recomputed: 1.4314452401187339e-12
Devitation between GF by propagating and by recomputed: 1.6782217976407665e-12
Devitation between GF by propagating and by recomputed: 1.2309959909057033e-12
Devitation between GF by propagating and by recomputed: 3.3979819825946667e-12
Devitation between GF by propagating and by recomputed: 1.4036867371572392e-13
Devitation between GF by propagating and by recomputed: 4.2140132529289165e-12
Devitation between GF by propagating and by recomputed: 1.121361196173426e-12
Devitation between GF by propagating and by recomputed: 7.940644669579555e-13
Devitation between GF by propagating and by recomputed: 2.5071453160080415e-12
Devitation between GF by propagating and by recomputed: 4.23698853779797e-11
Devitation between GF by propagating and by recomputed: 1.998973632021811e-12
Devitation between GF by propagating and by recomputed: 6.42118302973671e-13
Devitation between GF by propagating and by recomputed: 1.3

In [28]:
np.mean(sign)

1.0

In [29]:
(32-np.mean(Number)) - 16

4.073320012780613

In [33]:
Ntau = 30
mu = 1.7
sign, Number = run(N, Ntau, dtau, Nstable, U, t, mu, Nwarm, Nsweep)

Start warm up


Warming up:   0%|          | 0/500 [00:00<?, ?it/s]

Devitation between GF by propagating and by recomputed: 7.409836633165412e-13
Devitation between GF by propagating and by recomputed: 2.2792809306615425e-12
Devitation between GF by propagating and by recomputed: 1.3004874954702927e-12
Devitation between GF by propagating and by recomputed: 6.766920357392792e-12
Devitation between GF by propagating and by recomputed: 6.785801087705323e-12
Devitation between GF by propagating and by recomputed: 8.024809983186998e-12
Devitation between GF by propagating and by recomputed: 4.955665218464578e-12
Devitation between GF by propagating and by recomputed: 7.217552944194239e-12
Devitation between GF by propagating and by recomputed: 1.3068775228664009e-11
Devitation between GF by propagating and by recomputed: 1.6043136220542098e-11
Devitation between GF by propagating and by recomputed: 1.0222254466507597e-11
Devitation between GF by propagating and by recomputed: 1.0469625166820151e-11
Devitation between GF by propagating and by recomputed: 5.

Sweep:   0%|          | 0/2000 [00:00<?, ?it/s]

Devitation between GF by propagating and by recomputed: 5.460742968921295e-12
Devitation between GF by propagating and by recomputed: 6.885359470076846e-12
Devitation between GF by propagating and by recomputed: 1.1884382367099988e-11
Devitation between GF by propagating and by recomputed: 5.359497567969385e-13
Devitation between GF by propagating and by recomputed: 9.896637329087632e-12
Devitation between GF by propagating and by recomputed: 1.523776937961685e-11
Devitation between GF by propagating and by recomputed: 1.845787411802746e-12
Devitation between GF by propagating and by recomputed: 6.543026537242369e-12
Devitation between GF by propagating and by recomputed: 2.3076754049333736e-11
Devitation between GF by propagating and by recomputed: 6.900841009738201e-12
Devitation between GF by propagating and by recomputed: 2.831279655168828e-11
Devitation between GF by propagating and by recomputed: 6.332233348782523e-12
Devitation between GF by propagating and by recomputed: 1.6838

In [37]:
np.mean(sign * (32-Number))/np.mean(sign) - 16

4.663056344453711

In [35]:
np.mean(sign)

0.998

Ntau = 20, sign = 1, Number = 4.3